In [1]:
import numpy as np
import cv2
import random
import pandas as pd
import tensorflow as tf
import pickle
import ast
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from moviepy.editor import VideoFileClip

2023-10-21 13:42:31.690894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
good = '../dataset/error/good.csv'
misalignedHands = '../dataset/error/misaligned_hands.csv'
narrowLegs = '../dataset/error/narrow_legs.csv'

In [ ]:
df1 = pd.read_csv(good)
df2 = pd.read_csv(misalignedHands)
df3 = pd.read_csv(narrowLegs)

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df = pd.concat([df, df3], ignore_index=True)
df = df.drop(columns='Unnamed: 0', axis=1)
df

In [ ]:
newDataset = '../dataset/error/angles.csv'
df.to_csv(newDataset)

Chữa động tác

In [ ]:
df = pd.read_csv('../dataset/error/angles.csv')
df = df.drop('Unnamed: 0', axis=1)
# Convert the string representation of Angles into separate columns
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

# Convert the Phase column to integers
df['Phase'] = df['Phase'].astype(int)
df['Correction'] = df['Correction'].astype(int)
df.columns = df.columns.astype(str)
df

In [ ]:
features = df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']]
labels = df['Correction']

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)

In [ ]:
Ypred = clf.predict(Xtest)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy}")

In [ ]:
Ypred

In [ ]:
np.array(Ytest)

In [ ]:
Xtest = Xtest.sort_index()
Ytest = Ytest.sort_index()

In [ ]:
clf.predict(Xtest.loc[5:25])

In [ ]:
Xtest.loc[5:25]

In [ ]:
Ytest.loc[5:25]

Tính gia tốc, vận tốc

In [2]:
# Đếm số giây của video
videoPath = '../assets/sample-video/sample-video01.mp4'
video = VideoFileClip(videoPath)
duration = video.duration
duration

8.44

In [3]:
# Đếm số frame của video
cap = cv2.VideoCapture(videoPath)
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameCount

253

In [4]:
# Tính khoảng thời gian giữa 2 frame
deltaT = duration / (frameCount - 1)
deltaT

0.03349206349206349

In [5]:
df = pd.read_csv('../dataset/standard/angles01.csv')
df = df.drop('Unnamed: 0', axis=1)
df

,Angles,Phase,Score
0,"[1, 176, 177, 0, 180, 15, 177, 178]",Down,100
1,"[2, 176, 177, 0, 180, 14, 178, 178]",Down,100
2,"[2, 176, 177, 0, 180, 14, 179, 178]",Down,100
3,"[2, 176, 177, 0, 179, 14, 179, 178]",Down,100
4,"[2, 176, 177, 0, 179, 14, 179, 179]",Down,100
...,...,...,...
248,"[3, 175, 175, 0, 179, 14, 179, 179]",Down,100
249,"[3, 175, 175, 0, 179, 13, 179, 179]",Down,100
250,"[3, 175, 175, 0, 179, 14, 179, 179]",Down,100
251,"[3, 175, 175, 0, 179, 14, 179, 179]",Down,100


In [6]:
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

In [7]:
df.columns = df.columns.astype(str)

In [13]:
leftArms = df['1']

In [14]:
t = []
for i in range(len(leftArms)):
    t.append(deltaT * i)

In [19]:
x = np.array(leftArms)
t = np.array(t)

# Tính đạo hàm của mảng theo t
dx_dt = np.gradient(x, t)
dx_dt = dx_dt.astype(int)
print(dx_dt[0:75])

[    0     0     0     0     0     0     0     0     0     0     0   -14
   -14   -14   -14   -14   -14   -14   -14   -14   -14   -14   -59   -74
   -59   -59  -119  -298  -612 -1209 -1164  -507  -223  -134  -104   -59
   -29   -14   -14   -14   -14   -14   -14   -14   -14   -14     0     0
     0   -14   -14     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0    14    14     0
     0    14    14]


Chấm điểm

In [20]:
df = pd.read_csv('../dataset/error/angles.csv')
df = df.drop('Unnamed: 0', axis=1)
# Convert the string representation of Angles into separate columns
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

# Convert the Phase column to integers
df['Phase'] = df['Phase'].astype(int)
df['Score'] = df['Score'].astype(int)
df.columns = df.columns.astype(str)
df

,Phase,Correction,Score,0,1,2,3,4,5,6,7
0,1,0,100,1,176,177,0,180,15,177,178
1,1,0,100,2,176,177,0,180,14,178,178
2,1,0,100,2,176,177,0,180,14,179,178
3,1,0,100,2,176,177,0,179,14,179,178
4,1,0,100,2,176,177,0,179,14,179,179
...,...,...,...,...,...,...,...,...,...,...,...
842,2,9,70,3,40,45,0,180,8,179,173
843,2,9,70,3,39,44,0,179,8,179,174
844,2,9,70,4,39,44,0,179,8,179,174
845,2,9,70,3,38,43,0,179,8,180,174


In [21]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']])
labels = np.array(df['Score'])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

LR

In [22]:
model = LinearRegression()
reg = model.fit(Xtrain, Ytrain)
Ypred = reg.predict(Xtest)
Ypred = Ypred.astype(int)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 8.24%


In [23]:
Ypred

array([ 99,  97,  90,  95,  83,  74,  97,  77,  98,  80,  86,  99,  83,
        76,  82,  98,  72,  83,  94,  94,  89,  98,  97,  85,  94,  98,
        75,  96,  95,  98,  70,  95,  85,  97,  84,  71,  94,  97,  97,
        79,  87,  80,  96,  80,  79,  94,  98,  82,  85,  80,  87,  81,
        81,  99,  91,  77, 100,  85,  75,  81,  92,  71,  96, 100,  84,
        88,  98,  83,  72,  99,  96,  98,  76,  90,  44,  98,  72,  83,
        78,  90,  75,  82,  76,  68,  77,  96,  83,  96,  85,  80,  82,
        78,  97,  97, 100, 100,  95,  64,  98,  96, 107,  81,  99,  99,
        96,  70,  99,  85,  98,  99,  99,  76,  83,  90,  83,  83,  92,
        92,  82,  71,  77, 101,  95,  83,  87,  97,  98,  72,  99,  98,
        76,  79,  80,  99,  79,  97, 101,  72,  71,  98,  97, 100,  89,
       102,  85,  97,  97,  95,  97,  95,  93,  70,  95,  92,  83,  78,
        79,  79,  97,  85,  51,  71, 100, 101,  80,  98, 101, 100,  60,
        96])

In [24]:
Ytest

array([100,  80, 100, 100,  70,  70, 100,  70, 100,  70,  80, 100,  80,
        80,  80, 100,  70,  80,  85,  90, 100, 100, 100,  80,  90, 100,
        70, 100,  90, 100,  70, 100,  80, 100,  80,  70, 100, 100, 100,
        80,  85,  80, 100,  80,  70, 100, 100,  80,  95,  80,  80,  80,
        80, 100, 100,  80, 100,  95,  80,  80, 100,  80, 100, 100,  80,
       100, 100,  90,  80, 100, 100, 100,  80,  95,  60, 100,  70,  80,
        80, 100,  70,  85,  70,  70,  80, 100,  70, 100,  80,  80,  80,
        70, 100, 100, 100, 100,  90,  70, 100, 100,  95,  70, 100, 100,
       100,  70, 100,  80, 100, 100,  90,  70,  80, 100,  95,  80,  90,
        95,  80,  70,  80, 100, 100,  80,  80,  90, 100,  70, 100, 100,
        80,  80,  80, 100,  80, 100, 100,  80,  70, 100, 100,  90,  95,
       100,  80, 100, 100,  90, 100, 100,  90,  80,  90,  90,  80,  80,
        80,  80, 100,  80,  60,  70, 100, 100,  80,  90,  95, 100,  70,
       100])

SVR

In [25]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']])
labels = np.array(df['Score'])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [26]:
svr = svm.SVR(kernel='linear')
svr.fit(Xtrain, Ytrain)
Ypred = svr.predict(Xtest)
Ypred = Ypred.astype(int)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 8.82%


In [27]:
Ypred

array([101,  99,  93,  98,  85,  74,  98,  78,  98,  80,  85, 102,  85,
        78,  81,  99,  72,  82,  97,  97,  89, 100,  97,  85,  97,  98,
        74,  98,  98, 101,  69,  97,  84,  98,  84,  69,  96,  98,  99,
        78,  88,  78,  97,  79,  79,  96, 100,  81,  86,  82,  88,  80,
        81, 100,  91,  77, 102,  86,  75,  81,  93,  73,  97, 101,  84,
        91,  98,  84,  71, 101,  97, 100,  76,  91,  43, 100,  71,  83,
        78,  91,  75,  82,  79,  67,  78,  98,  85,  98,  86,  80,  84,
        80,  97,  99, 102, 101,  97,  62, 100,  96, 110,  81, 101, 100,
        98,  69, 100,  86,  99, 100, 101,  75,  84,  93,  84,  83,  93,
        94,  81,  71,  76, 101,  97,  85,  87,  99,  98,  71,  99,  98,
        75,  79,  79,  99,  78,  99, 101,  73,  72,  99,  98, 102,  91,
       103,  85,  99,  97,  97,  99,  97,  95,  69,  97,  95,  82,  76,
        80,  80,  98,  85,  49,  72, 100, 101,  81,  98, 103, 100,  58,
        98])

In [28]:
Ytest

array([100,  80, 100, 100,  70,  70, 100,  70, 100,  70,  80, 100,  80,
        80,  80, 100,  70,  80,  85,  90, 100, 100, 100,  80,  90, 100,
        70, 100,  90, 100,  70, 100,  80, 100,  80,  70, 100, 100, 100,
        80,  85,  80, 100,  80,  70, 100, 100,  80,  95,  80,  80,  80,
        80, 100, 100,  80, 100,  95,  80,  80, 100,  80, 100, 100,  80,
       100, 100,  90,  80, 100, 100, 100,  80,  95,  60, 100,  70,  80,
        80, 100,  70,  85,  70,  70,  80, 100,  70, 100,  80,  80,  80,
        70, 100, 100, 100, 100,  90,  70, 100, 100,  95,  70, 100, 100,
       100,  70, 100,  80, 100, 100,  90,  70,  80, 100,  95,  80,  90,
        95,  80,  70,  80, 100, 100,  80,  80,  90, 100,  70, 100, 100,
        80,  80,  80, 100,  80, 100, 100,  80,  70, 100, 100,  90,  95,
       100,  80, 100, 100,  90, 100, 100,  90,  80,  90,  90,  80,  80,
        80,  80, 100,  80,  60,  70, 100, 100,  80,  90,  95, 100,  70,
       100])

TensorFlow

In [29]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']].values)
labels = np.array(df['Score'].values)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [30]:
# Chuẩn hoá dữ liệu
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [31]:
# Xây dựng mô hình mạng neural
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(9,)),  # Số lượng đặc trưng đầu vào
    tf.keras.layers.Dense(64, activation='relu'),  # Lớp ẩn với 64 đơn vị ẩn
    tf.keras.layers.Dense(32, activation='relu'),  # Lớp ẩn với 32 đơn vị ẩn
    tf.keras.layers.Dense(1)  # Lớp đầu ra với 1 đơn vị (Regression)
])

In [32]:
# Biên dịch mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(Xtrain, Ytrain, epochs=50, batch_size=32, validation_data=(Xtest, Ytest))

# Đánh giá hiệu suất của mô hình
loss = model.evaluate(Xtest, Ytest)
print(f"Loss on test data: {loss}")

Epoch 1/50
22/22 [==============================] - 1s 12ms/step - loss: 7971.6978 - val_loss: 7672.9043
Epoch 2/50
22/22 [==============================] - 0s 3ms/step - loss: 7715.2422 - val_loss: 7345.2256
Epoch 3/50
22/22 [==============================] - 0s 3ms/step - loss: 7288.1719 - val_loss: 6775.4360
Epoch 4/50
22/22 [==============================] - 0s 3ms/step - loss: 6557.5630 - val_loss: 5844.3804
Epoch 5/50
22/22 [==============================] - 0s 3ms/step - loss: 5441.2700 - val_loss: 4540.1328
Epoch 6/50
22/22 [==============================] - 0s 3ms/step - loss: 4002.0110 - val_loss: 3013.0918
Epoch 7/50
22/22 [==============================] - 0s 3ms/step - loss: 2460.3015 - val_loss: 1591.0513
Epoch 8/50
22/22 [==============================] - 0s 3ms/step - loss: 1200.8969 - val_loss: 666.5773
Epoch 9/50
22/22 [==============================] - 0s 4ms/step - loss: 507.9108 - val_loss: 297.9973
Epoch 10/50
22/22 [==============================] - 0s 4ms/step -

In [33]:
# Sử dụng mô hình để dự đoán
predictions = model.predict(Xtest)

6/6 [==============================] - 0s 1ms/step


In [36]:
Ypred = predictions.astype(int)
Ypred = predictions.reshape(-1,)
Ypred

array([ 97.4832  ,  92.58775 ,  95.96651 ,  92.38681 ,  78.58664 ,
        69.10091 , 108.18225 ,  70.33825 ,  96.50507 ,  62.355316,
        82.09651 , 106.298965,  77.56454 ,  77.97844 ,  81.47011 ,
        93.5946  ,  83.630295,  83.84516 ,  82.45807 ,  96.245865,
        90.845726,  81.84943 ,  92.23066 ,  71.35959 ,  97.61862 ,
       101.4261  ,  72.76367 , 100.405075,  87.559006, 104.929855,
        82.833206, 101.97603 ,  80.836845,  96.75665 ,  79.04896 ,
        79.39    , 100.329155,  94.75281 ,  96.66629 ,  78.10262 ,
        77.663345,  80.92355 ,  95.40703 ,  82.518715,  77.36655 ,
        97.59041 , 105.11337 ,  79.94606 ,  87.08876 ,  74.56408 ,
        78.308914,  76.19464 ,  81.83257 , 102.46239 ,  96.30831 ,
        68.54332 ,  99.728226,  85.13704 ,  84.49437 ,  82.19505 ,
        96.05626 ,  75.66583 , 103.799164, 103.34517 ,  81.18147 ,
       101.783295,  92.56947 ,  92.300766,  80.281525,  97.4832  ,
        96.681984, 105.11337 ,  84.638664,  96.17323 ,  71.778

In [37]:
Ytest

array([100,  80, 100, 100,  70,  70, 100,  70, 100,  70,  80, 100,  80,
        80,  80, 100,  70,  80,  85,  90, 100, 100, 100,  80,  90, 100,
        70, 100,  90, 100,  70, 100,  80, 100,  80,  70, 100, 100, 100,
        80,  85,  80, 100,  80,  70, 100, 100,  80,  95,  80,  80,  80,
        80, 100, 100,  80, 100,  95,  80,  80, 100,  80, 100, 100,  80,
       100, 100,  90,  80, 100, 100, 100,  80,  95,  60, 100,  70,  80,
        80, 100,  70,  85,  70,  70,  80, 100,  70, 100,  80,  80,  80,
        70, 100, 100, 100, 100,  90,  70, 100, 100,  95,  70, 100, 100,
       100,  70, 100,  80, 100, 100,  90,  70,  80, 100,  95,  80,  90,
        95,  80,  70,  80, 100, 100,  80,  80,  90, 100,  70, 100, 100,
        80,  80,  80, 100,  80, 100, 100,  80,  70, 100, 100,  90,  95,
       100,  80, 100, 100,  90, 100, 100,  90,  80,  90,  90,  80,  80,
        80,  80, 100,  80,  60,  70, 100, 100,  80,  90,  95, 100,  70,
       100])

In [38]:
# Tính MSE
mse = mean_squared_error(Ytest, Ypred)
print(f"Mean Squared Error (MSE): {mse}")

# Tính RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Squared Error (MSE): 36.93240263405511
Root Mean Squared Error (RMSE): 6.077203520868386


In [39]:
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')

Accuracy: 58.82%


Random Forest

In [40]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']].values)
labels = np.array(df['Score'].values)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [41]:
# Tạo mô hình Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # Bạn có thể tùy chỉnh số cây và các siêu tham số khác

# Huấn luyện mô hình trên tập huấn luyện
rf_model.fit(Xtrain, Ytrain)

# Dự đoán trên tập kiểm tra
Ypred = rf_model.predict(Xtest)

# Đánh giá mô hình
mse = mean_squared_error(Ytest, Ypred)
r2 = r2_score(Ytest, Ypred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 3.1750045955882356
R-squared (R2): 0.976485214386392


In [42]:
Ypred = Ypred.astype(int)
Ypred

array([100,  91,  99, 100,  75,  70, 100,  75, 100,  71,  81, 100,  80,
        79,  80, 100,  66,  80,  85,  90,  97,  99,  96,  80,  91,  98,
        71,  99,  88, 100,  67, 100,  80, 100,  80,  67, 100,  98, 100,
        80,  84,  80, 100,  80,  71, 100, 100,  80,  95,  79,  80,  80,
        80, 100,  99,  80, 100,  94,  79,  80,  99,  79,  98, 100,  80,
        97,  96,  90,  78, 100, 100, 100,  80,  94,  61, 100,  70,  80,
        79,  98,  70,  81,  69,  71,  79, 100,  75, 100,  76,  80,  73,
        74, 100, 100,  99, 100,  90,  72, 100, 100,  95,  71, 100, 100,
       100,  73, 100,  80,  99, 100,  90,  70,  80,  99,  88,  80,  90,
        96,  79,  70,  80, 100, 100,  80,  80,  90,  97,  72,  99, 100,
        80,  80,  80,  98,  79, 100, 100,  80,  70,  99, 100,  90,  90,
        97,  80, 100, 100,  90, 100, 100,  90,  80,  88,  90,  80,  80,
        79,  79, 100,  80,  62,  70, 100, 100,  79,  92,  95, 100,  72,
       100])

In [43]:
Ytest

array([100,  80, 100, 100,  70,  70, 100,  70, 100,  70,  80, 100,  80,
        80,  80, 100,  70,  80,  85,  90, 100, 100, 100,  80,  90, 100,
        70, 100,  90, 100,  70, 100,  80, 100,  80,  70, 100, 100, 100,
        80,  85,  80, 100,  80,  70, 100, 100,  80,  95,  80,  80,  80,
        80, 100, 100,  80, 100,  95,  80,  80, 100,  80, 100, 100,  80,
       100, 100,  90,  80, 100, 100, 100,  80,  95,  60, 100,  70,  80,
        80, 100,  70,  85,  70,  70,  80, 100,  70, 100,  80,  80,  80,
        70, 100, 100, 100, 100,  90,  70, 100, 100,  95,  70, 100, 100,
       100,  70, 100,  80, 100, 100,  90,  70,  80, 100,  95,  80,  90,
        95,  80,  70,  80, 100, 100,  80,  80,  90, 100,  70, 100, 100,
        80,  80,  80, 100,  80, 100, 100,  80,  70, 100, 100,  90,  95,
       100,  80, 100, 100,  90, 100, 100,  90,  80,  90,  90,  80,  80,
        80,  80, 100,  80,  60,  70, 100, 100,  80,  90,  95, 100,  70,
       100])

In [44]:
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')

Accuracy: 98.24%
